In [ ]:
from dotenv import load_dotenv
load_dotenv()
from langchain.chat_models import init_chat_model
from langchain_google_firestore import FirestoreVectorStore
from langchain_google_vertexai import VertexAIEmbeddings,ChatVertexAI
from langchain_core.tools.retriever import create_retriever_tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

try:
    print("1. Inicializando componentes...")
    llm = ChatVertexAI(model_name="gemini-2.0-flash-001") # Usando un modelo más reciente por si acaso
    FIRESTORE_COLLECTION_NAME = "knowledge_base"
    embeddings = VertexAIEmbeddings(
        model_name="text-embedding-004", # Usando un modelo estable
        location="us-central1"
    )
    vector_store = FirestoreVectorStore(
        collection=FIRESTORE_COLLECTION_NAME, embedding_service=embeddings
    )
    retriever = vector_store.as_retriever(search_kwargs={"k": 3})
    print("   ...Componentes inicializados con éxito.")

except Exception as e:
    print(f"❌ ¡Ocurrió un error durante la inicialización!: {e}")
    # Detenemos la ejecución si la inicialización falla
    exit()

# --- DIAGNÓSTICO CLAVE: Probar el Retriever por separado ---
print("\n2. Probando el Retriever de forma aislada...")
pregunta = "¿Quien es edgardo ponce?"
try:
    documentos_encontrados = retriever.invoke(pregunta)
    print("--- SALIDA DEL RETRIEVER ---")
    if not documentos_encontrados:
        print("‼️  ATENCIÓN: El retriever NO encontró ningún documento para la pregunta.")
        print("   Esta es la causa más probable de que no obtengas respuesta.")
    else:
        print(f"✅  El retriever encontró {len(documentos_encontrados)} documento(s).")
        for i, doc in enumerate(documentos_encontrados):
            print(f"   --- Documento {i+1} ---")
            print(f"   Contenido: {doc.page_content[:200]}...") # Muestra los primeros 200 caracteres
            print(f"   Metadata: {doc.metadata}")
            print("-" * 20)
except Exception as e:
    print(f"❌ ¡Ocurrió un error al usar el retriever!: {e}")


# --- 3. Creación de la Cadena (sin cambios aquí) ---
print("\n3. Construyendo la cadena RAG...")
retriever_tool = create_retriever_tool(
    retriever,
    "busqueda_en_base_de_conocimiento",
    "Busca y devuelve información relevante desde la base de datos de documentos."
)
llm_with_tools = llm.bind_tools([retriever_tool])
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Eres un asistente que responde preguntas basándose únicamente en el contexto proporcionado. Contexto:\n\n{context}",
        ),
        ("human", "{question}"),
    ]
)

# --- 4. Modificación de la Cadena para ser "Locuaz" ---
# Vamos a añadir un paso para poder ver el contexto que se está usando
def format_docs(docs):
    print("\n--- DEBUG: Formateando documentos para el prompt ---")
    if not docs:
        print("   Contexto está vacío porque no se encontraron documentos.")
        return "No se encontró información relevante."
    formatted_context = "\n\n".join(doc.page_content for doc in docs)
    print(f"   Contexto a enviar al LLM: {formatted_context[:300]}...")
    return formatted_context

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# --- 5. Invocación de la Cadena ---
print("\n4. Invocando la cadena RAG completa...")
try:
    respuesta = rag_chain.invoke(pregunta)

    print("\n--- RESPUESTA FINAL ---")
    if respuesta:
        print(respuesta)
    else:
        print("‼️  La cadena se ejecutó pero devolvió una respuesta vacía.")

except Exception as e:
    print(f"❌ ¡Ocurrió un error al invocar la cadena RAG!: {e}")




1. Inicializando componentes...


E0000 00:00:1759949192.640432   75912 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
/home/eponce/Tools/ia/lang-graph/.venv/lib/python3.12/site-packages/vertexai/_model_garden/_model_garden_models.py:278: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()
E0000 00:00:1759949193.887361   75912 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


   ...Componentes inicializados con éxito.

2. Probando el Retriever de forma aislada...


E0000 00:00:1759949195.468780   76107 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
E0000 00:00:1759949195.470557   76107 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
E0000 00:00:1759949196.555335   75912 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


--- SALIDA DEL RETRIEVER ---
✅  El retriever encontró 3 documento(s).
   --- Documento 1 ---
   Contenido: ## 📫 How to Reach Me  
You can find me on the following channels. I'm open to collaborating on interesting freelance projects!  
- **LinkedIn:** [Edgardo Ponce](https://www.linkedin.com/in/edgardo-pon...
   Metadata: {'reference': {'path': 'knowledge_base/dHFHu66Fg3KgIJKWiiO1', 'firestore_type': 'document_reference'}, 'embedding': {'values': [0.006884101312607527, -0.03959829360246658, -0.09543143212795258, 0.035183679312467575, 0.04856064170598984, 0.017447272315621376, 0.042495377361774445, -0.00017913714691530913, 0.05935044586658478, -0.009438034147024155, -0.03948148339986801, 0.026481863111257553, 0.02731422521173954, 0.011293021962046623, -0.007968645542860031, -0.042799487709999084, 0.01930491253733635, 0.020735226571559906, -0.10500053316354752, 0.003868142142891884, 0.06739853322505951, -0.032760459929704666, 0.020022185519337654, -0.00687593687325716, -0.027098616585135

E0000 00:00:1759949200.052378   75912 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.



--- RESPUESTA FINAL ---
Edgardo Ponce es un Analista Programador, Especialista en IA y Automatización, y Freelancer. Es un Desarrollador graduado de la Universidad Tecnológica Nacional (UTN) con pasión por resolver problemas complejos a través del código.
